# Introduing $\texttt{NatPy}$, a simple and convenient python module for dealing with and converting natural units.
## Requires $\texttt{python>=3.7 }$, $\texttt{astropy}$

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/AndreScaffidi/Natpy_pyhep_2021/HEAD?filepath=pyhep.ipynb)


In [ ]:
!pip install natpy

## Ever wanted to convert MeV / fm$^{3}$ to Pascals?

In [ ]:
import natpy as nat
print("Before conversion:")
a = 1 * nat.MeV * nat.fm**(-3)
a

In [ ]:
print("After conversion:")
a.convert(nat.Pa)

## Standard HEP unit conventions require 
$\bar{h} = c = \epsilon_0=1$ 
* $\alpha=\frac{e^{2}}{ 4 \pi \varepsilon_{0} \hbar c }\simeq 1 / 137$
* $e=\sqrt{4 \pi \alpha} \simeq 0.303$
* Mass, momentum, energy are measured in GeV $(m=m_{\mathrm{NU}} / c^{2}, p=p_{\mathrm{NU}} / c \text { ) }$
* Length is measured in GeV$^{-1}\left(L_{\mathrm{SI}}=L_{\mathrm{NU}} \times \hbar c\right)$
* Cross section (area) is measured in GeV$^{-2}\left(\sigma_{\mathrm{SI}}=\sigma_{\mathrm{NU}} \times(\hbar c)^{2}\right)$
* Time is measured in GeV$^{-1}\left(t_{\mathrm{SI}}=t_{\mathrm{NU}} \times \hbar\right)$

## NatPy quantities are astropy.units.quantity.Quantity objects

In [ ]:
a = 1 * nat.GeV
type(a)

### Can access astropy quantities easily

In [ ]:
nat.const.c

In [ ]:
nat.const.hbar

In [ ]:
nat.const.e

$\texttt{NatPy}$: Convert $e$ to natural units

In [ ]:
nat.const.e.convert(nat.one)

## $\texttt{Natpy}$ real world workflow example
Consider calculating an average decay length for some new physics: 

$$
\bar{L} = \beta\gamma\left(\frac{1}{\Gamma}\right)
$$
where

$$
\Gamma\left(\chi_{2} \rightarrow \chi_{1} \ell^{+} \ell^{-}\right) = \frac{4 \epsilon^{2} \alpha_{\mathrm{em}} \alpha_{D} \Delta^{5} m_{1}^{5}}{15 \pi m_{A^{\prime}}^{4}}
$$

In [ ]:
# All unitful physical quantities handled by NatPy!
import numpy as np
m1    = 10 * nat.GeV
mA    = 3 * m1
Delta = 0.1
m2    = Delta*m1 + m1
epsi  = 5.7E-5
alphaD= 0.1

In [ ]:
# Decay width
def Gamma(ep,m_1,m_A,alpha_D,delta):
    numerator   = 4 * ep**2 * nat.const.alpha * alpha_D * delta**5 * m_1**5
    denominator = 15 * np.pi * m_A**4
    return numerator / denominator
# Decay length
def boost_factor(m,p):
    E = np.sqrt(p**2+m**2)
    return (E/m)
def Beta(boost_factor): 
    return np.sqrt(1 - 1/boost_factor**2)
def L_bar(p):
    Chi2_gamma  = boost_factor(m2,p)
    d           = Beta(Chi2_gamma)*Chi2_gamma*(1 / Gamma(epsi,m1,mA,alphaD,Delta)) 
    return d 
# Say particle has |p| = 7 GeV... 
print("Calling L_bar() gives use an object with units 1 / GeV")
L_bar(7 * nat.GeV)

In [ ]:
print("Post conversion")
L_bar(7 * nat.GeV).convert(nat.m) # Convert everything to meters at the end!

**If intrinsic (natural) unit consistency is achieved, can always convert the final result to whatever units we like!!**

### **Spicy**: Input  $m_1$ in kg?!

In [ ]:
m1 = m1.convert(nat.kg)
m1

## Changing unit systems

In [ ]:
nat.set_active_units("planck")
print("Planck length in m")
nat.one.convert(nat.m)

### $\texttt{Natpy}$ under the hood

![title](img/equation_natpy.png)

The difference in dimentionallity is wrapped up in $\bar{d}$

$$\bar{d} = \hbar^{n_{\hbar}}c^{n_c}...$$

$$\text { Define } 
\;\vec{d}=\left(\begin{array}{c}
n_{\hbar} \\
n_{c} \\
\vdots
\end{array}\right)$$

$\Rightarrow$ linear algebra problem! After a little math...

$$
\vec{d}
=
\begin{pmatrix}
 & \text{Psuedo inverse matrix involving} &  \\ 
 & \text{powers of mass/time/length} &  \\
 & \text{for } \bar{q}_f\text{ and }\bar{q_i} &  \\
\end{pmatrix}              
$$
